# Running processes same as DataWrangling11 on all counties available in US

# Clean up new added datasets unemployment rates, poverty rate, education and vehicle

# Importing libraries

In [196]:
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import klib

from IPython.core.display import display

# pd.set_option('display.max_columns', None)
# pd.reset_optio('max_rows')
#np.set_printoptions(threshold=sys.maxsize)

plt.style.use('dark_background')
plt.rcParams.update({"grid.linewidth":0.5, "grid.alpha":0.5})
sns.set(style='ticks', context='talk')

# Load data and constants

In [197]:
daily_aqi_by_county_2017 = pd.read_csv('../../data/raw/daily_aqi_by_county_2017.csv')
daily_aqi_by_county_2018 = pd.read_csv('../../data/raw/daily_aqi_by_county_2018.csv')
daily_aqi_by_county_2019 = pd.read_csv('../../data/raw/daily_aqi_by_county_2019.csv')

county_demographic = pd.read_csv('../../data/raw/county_cc-est2019-alldata.csv', encoding='latin-1')

unemployment = pd.read_excel('../../data/raw/Unemployment.xls', header=None, sheet_name=0)

# Small area income and poverty estimates
saipe2017 = pd.read_excel('../../data/raw/saipe_economic/est17all.xls')
saipe2018 = pd.read_excel('../../data/raw/saipe_economic/est18all.xls')
saipe2019 = pd.read_excel('../../data/raw/saipe_economic/est19all.xls')

educ_vehicle_2017 = pd.read_csv('../../data/processed/acs1_edu_vehicle2017.csv')
educ_vehicle_2018 = pd.read_csv('../../data/processed/acs1_edu_vehicle2018.csv')
educ_vehicle_2019 = pd.read_csv('../../data/processed/acs1_edu_vehicle2019.csv')

annual_income_by_county = pd.read_csv('../../data/raw/CAINC1__ALL_AREAS_1969_2019.csv', encoding='latin-1')


In [198]:
def remove_col(df, col_name):
    '''Returns a dataframe with removed column or columns from old dataframe'''
    new_df = df.copy()

    if(type(col_name) != str and len(col_name) > 1):
        for index in col_name:
            new_df = new_df.drop(str(index), axis=1)
    else:
        new_df = new_df.drop(str(col_name), axis=1)
    return new_df

def get_df_with_geofips(df, state_code, county_code):
    ''' Returns dataframe with geofips column '''
    state_code = df[state_code].astype(str).values
    county_code = df[county_code].astype(str).values

    geofips = []
    for stateID, countyID in zip(state_code, county_code):
        id = ''
        stateLen = len(stateID)
        countyLen = len(countyID)
        if(stateLen < 2):
            id += '0' + stateID
        else:
            id += stateID
        if(countyLen == 1):
            id += '00' + countyID
        elif(countyLen == 2):
            id += '0' + countyID
        else:
            id += countyID
        geofips.append(id)

    df.insert(loc=0, column='GeoFIPS', value=geofips)
    return df

## Clean up and set up income datasets


In [199]:
annual_income_by_county

all_incomes_2017_2019 = remove_col(annual_income_by_county, range(1969, 2017))

# used to remove last 4 rows as they are not index data; looks like extra info
all_incomes_2017_2019 = all_incomes_2017_2019[:len(all_incomes_2017_2019) - 4]

# all_incomes_2017_2019.info()
all_incomes_2017_2019

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2017,2018,2019
0,"""00000""",United States,,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,16937582000,17839255000,18542262000
1,"""00000""",United States,,CAINC1,2.0,...,Population (persons) 1/,Number of persons,324985539,326687501,328239523
2,"""00000""",United States,,CAINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,52118,54606,56490
3,"""01000""",Alabama,5,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,199999756,208752683,216449038
4,"""01000""",Alabama,5,CAINC1,2.0,...,Population (persons) 1/,Number of persons,4874486,4887681,4903185
...,...,...,...,...,...,...,...,...,...,...,...
9589,"""97000""",Rocky Mountain,7,CAINC1,2.0,...,Population (persons) 1/,Number of persons,12062055,12233639,12399296
9590,"""97000""",Rocky Mountain,7,CAINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,49991,52936,54873
9591,"""98000""",Far West,8,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,3279057722,3472097346,3634009401
9592,"""98000""",Far West,8,CAINC1,2.0,...,Population (persons) 1/,Number of persons,56059482,56350416,56572426


* Years 1969 to 2016 were removed as we only have interest on 2017-2019
* Also last four rows were remoed as they were not row data, there were just caption text


In [200]:
all_incomes_2017_2019['2017'] = pd.to_numeric(all_incomes_2017_2019['2017'], downcast='float', errors='coerce')
all_incomes_2017_2019['2018'] = pd.to_numeric(all_incomes_2017_2019['2018'], downcast='float', errors='coerce')
all_incomes_2017_2019['2019'] = pd.to_numeric(all_incomes_2017_2019['2019'], downcast='float', errors='coerce')

# all_incomes_2017_2019 = remove_col(all_incomes_2017_2019, range(2017,2020))
all_incomes_2017_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9594 entries, 0 to 9593
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   GeoFIPS                 9594 non-null   object 
 1   GeoName                 9594 non-null   object 
 2   Region                  9594 non-null   object 
 3   TableName               9594 non-null   object 
 4   LineCode                9594 non-null   float64
 5   IndustryClassification  9594 non-null   object 
 6   Description             9594 non-null   object 
 7   Unit                    9594 non-null   object 
 8   2017                    9519 non-null   float32
 9   2018                    9519 non-null   float32
 10  2019                    9519 non-null   float32
dtypes: float32(3), float64(1), object(7)
memory usage: 712.2+ KB


* Columns were changed to appropriate types
* Income was changed to type float
* 'NA' string was changed to np.nan
** Rename column income data to be more descriptive and remove old column in place of new column names

In [201]:
personal_income = all_incomes_2017_2019['Description'] == ('Personal income (thousands of dollars)')
per_capita_personal_income = all_incomes_2017_2019['Description'] == ('Per capita personal income (dollars) 2/')
county_personal_incomes = all_incomes_2017_2019[personal_income]
county_per_capita_personal_income = all_incomes_2017_2019[per_capita_personal_income]
county_personal_incomes.rename(columns={'2017':'Personal income (thousands of dollars) 2017',
                                        '2018':'Personal income (thousands of dollars) 2018',
                                        '2019':'Personal income (thousands of dollars) 2019',}, inplace=True)
county_per_capita_personal_income.rename(columns={'2017':'Per capita personal income (dollars) 2017',
                                                  '2018':'Per capita personal income (dollars) 2018',
                                                  '2019':'Per capita personal income (dollars) 2019',}, inplace=True)

display(county_personal_incomes)
display(county_per_capita_personal_income)

/home/wkc/.local/lib/python3.8/site-packages/pandas/core/frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,Personal income (thousands of dollars) 2017,Personal income (thousands of dollars) 2018,Personal income (thousands of dollars) 2019
0,"""00000""",United States,,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,1.693758e+10,1.783926e+10,1.854226e+10
3,"""01000""",Alabama,5,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,1.999998e+08,2.087527e+08,2.164490e+08
6,"""01001""","Autauga, AL",5,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,2.276561e+06,2.360366e+06,2.453617e+06
9,"""01003""","Baldwin, AL",5,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,9.471242e+06,1.006597e+07,1.060026e+07
12,"""01005""","Barbour, AL",5,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,8.381840e+05,8.721890e+05,8.828340e+05
...,...,...,...,...,...,...,...,...,...,...,...
9579,"""94000""",Plains,4,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,1.055029e+09,1.107630e+09,1.146515e+09
9582,"""95000""",Southeast,5,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,3.818755e+09,4.022276e+09,4.173677e+09
9585,"""96000""",Southwest,6,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,1.924648e+09,2.051027e+09,2.144764e+09
9588,"""97000""",Rocky Mountain,7,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,6.029942e+08,6.476007e+08,6.803901e+08


,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,Per capita personal income (dollars) 2017,Per capita personal income (dollars) 2018,Per capita personal income (dollars) 2019
2,"""00000""",United States,,CAINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,52118.0,54606.0,56490.0
5,"""01000""",Alabama,5,CAINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,41030.0,42710.0,44145.0
8,"""01001""","Autauga, AL",5,CAINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,41101.0,42504.0,43917.0
11,"""01003""","Baldwin, AL",5,CAINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,44566.0,46205.0,47485.0
14,"""01005""","Barbour, AL",5,CAINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,33318.0,35067.0,35763.0
...,...,...,...,...,...,...,...,...,...,...,...
9581,"""94000""",Plains,4,CAINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,49609.0,51879.0,53509.0
9584,"""95000""",Southeast,5,CAINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,45657.0,47715.0,49159.0
9587,"""96000""",Southwest,6,CAINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,46531.0,49045.0,50670.0
9590,"""97000""",Rocky Mountain,7,CAINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,49991.0,52936.0,54873.0


In [202]:
cols_to_remove = ['IndustryClassification', 'TableName', 'Unit', 'Description', 'LineCode', 'Region']

county_personal_incomes.drop(cols_to_remove, axis=1, inplace=True)
county_per_capita_personal_income.drop(cols_to_remove, axis=1, inplace=True)

county_incomes = pd.merge(county_personal_incomes, county_per_capita_personal_income, how='inner')
county_incomes

/home/wkc/.local/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,GeoFIPS,GeoName,Personal income (thousands of dollars) 2017,Personal income (thousands of dollars) 2018,Personal income (thousands of dollars) 2019,Per capita personal income (dollars) 2017,Per capita personal income (dollars) 2018,Per capita personal income (dollars) 2019
0,"""00000""",United States,1.693758e+10,1.783926e+10,1.854226e+10,52118.0,54606.0,56490.0
1,"""01000""",Alabama,1.999998e+08,2.087527e+08,2.164490e+08,41030.0,42710.0,44145.0
2,"""01001""","Autauga, AL",2.276561e+06,2.360366e+06,2.453617e+06,41101.0,42504.0,43917.0
3,"""01003""","Baldwin, AL",9.471242e+06,1.006597e+07,1.060026e+07,44566.0,46205.0,47485.0
4,"""01005""","Barbour, AL",8.381840e+05,8.721890e+05,8.828340e+05,33318.0,35067.0,35763.0
...,...,...,...,...,...,...,...,...
3193,"""94000""",Plains,1.055029e+09,1.107630e+09,1.146515e+09,49609.0,51879.0,53509.0
3194,"""95000""",Southeast,3.818755e+09,4.022276e+09,4.173677e+09,45657.0,47715.0,49159.0
3195,"""96000""",Southwest,1.924648e+09,2.051027e+09,2.144764e+09,46531.0,49045.0,50670.0
3196,"""97000""",Rocky Mountain,6.029942e+08,6.476007e+08,6.803901e+08,49991.0,52936.0,54873.0


Removed redundant or uninterested columns
* remove more columns
* remove IndustryClassifaction as it only contains ... values and also not what we are interested in
* remove TableName as only one value and not what we are interested in
* remove Unit as that has the same value as Thousands of dollars
* remove Description as that is all personal income now
* remove LineCode as there are no unique values all value is 1
* remove Region as not interested in this data

In [203]:
county_incomes['GeoFIPS'] = county_incomes['GeoFIPS'].str.strip('" "')
county_incomes

,GeoFIPS,GeoName,Personal income (thousands of dollars) 2017,Personal income (thousands of dollars) 2018,Personal income (thousands of dollars) 2019,Per capita personal income (dollars) 2017,Per capita personal income (dollars) 2018,Per capita personal income (dollars) 2019
0,00000,United States,1.693758e+10,1.783926e+10,1.854226e+10,52118.0,54606.0,56490.0
1,01000,Alabama,1.999998e+08,2.087527e+08,2.164490e+08,41030.0,42710.0,44145.0
2,01001,"Autauga, AL",2.276561e+06,2.360366e+06,2.453617e+06,41101.0,42504.0,43917.0
3,01003,"Baldwin, AL",9.471242e+06,1.006597e+07,1.060026e+07,44566.0,46205.0,47485.0
4,01005,"Barbour, AL",8.381840e+05,8.721890e+05,8.828340e+05,33318.0,35067.0,35763.0
...,...,...,...,...,...,...,...,...
3193,94000,Plains,1.055029e+09,1.107630e+09,1.146515e+09,49609.0,51879.0,53509.0
3194,95000,Southeast,3.818755e+09,4.022276e+09,4.173677e+09,45657.0,47715.0,49159.0
3195,96000,Southwest,1.924648e+09,2.051027e+09,2.144764e+09,46531.0,49045.0,50670.0
3196,97000,Rocky Mountain,6.029942e+08,6.476007e+08,6.803901e+08,49991.0,52936.0,54873.0


* remove "" characters from income dataset


In [204]:

county_incomes_2017_2019 = county_incomes
county_incomes_2017_2019['GeoFIPS'] = pd.to_numeric(county_incomes_2017_2019['GeoFIPS'])
county_incomes_2017_2019

,GeoFIPS,GeoName,Personal income (thousands of dollars) 2017,Personal income (thousands of dollars) 2018,Personal income (thousands of dollars) 2019,Per capita personal income (dollars) 2017,Per capita personal income (dollars) 2018,Per capita personal income (dollars) 2019
0,0,United States,1.693758e+10,1.783926e+10,1.854226e+10,52118.0,54606.0,56490.0
1,1000,Alabama,1.999998e+08,2.087527e+08,2.164490e+08,41030.0,42710.0,44145.0
2,1001,"Autauga, AL",2.276561e+06,2.360366e+06,2.453617e+06,41101.0,42504.0,43917.0
3,1003,"Baldwin, AL",9.471242e+06,1.006597e+07,1.060026e+07,44566.0,46205.0,47485.0
4,1005,"Barbour, AL",8.381840e+05,8.721890e+05,8.828340e+05,33318.0,35067.0,35763.0
...,...,...,...,...,...,...,...,...
3193,94000,Plains,1.055029e+09,1.107630e+09,1.146515e+09,49609.0,51879.0,53509.0
3194,95000,Southeast,3.818755e+09,4.022276e+09,4.173677e+09,45657.0,47715.0,49159.0
3195,96000,Southwest,1.924648e+09,2.051027e+09,2.144764e+09,46531.0,49045.0,50670.0
3196,97000,Rocky Mountain,6.029942e+08,6.476007e+08,6.803901e+08,49991.0,52936.0,54873.0


* Retrieve NE county income data
* convert GeoFIPS to numeric type


# Clean up Unemployment Data


In [205]:
display(unemployment)
unemployment2017_2019 = unemployment[4:]
unemployment2017_2019.reset_index(drop=True, inplace=True)
unemployment_header = unemployment2017_2019.iloc[0, :].values
unemployment2017_2019.columns = unemployment_header
unemployment2017_2019 = unemployment2017_2019.drop([0])
display(unemployment2017_2019)

,0,1,2,3,4,5,6,7,8,9,...,78,79,80,81,82,83,84,85,86,87
0,Unemployment and median household income for t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sources: Unemployment: U.S. Department of Labo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"For definitions of rural classifications, see ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"This table was prepared by USDA, Economic Rese...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,fips_txt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3275,72145,PR,"Vega Baja Municipio, PR",1,1,1,19805,17607,2198,11.1,...,13117,11750,1367,10.4,13037,11791,1246,9.6,NaN,NaN
3276,72147,PR,"Vieques Municipio, PR",7,12,0,2441,2120,321,13.2,...,2718,2269,449,16.5,2585,2406,179,6.9,NaN,NaN
3277,72149,PR,"Villalba Municipio, PR",2,2,1,7636,6648,988,12.9,...,7311,6154,1157,15.8,7406,6231,1175,15.9,NaN,NaN
3278,72151,PR,"Yabucoa Municipio, PR",1,1,1,10613,9005,1608,15.2,...,8762,7509,1253,14.3,8691,7552,1139,13.1,NaN,NaN


,fips_txt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
1,00000,US,United States,NaN,NaN,NaN,142601667,136904680,5696987,3.99504,...,161389026,155102319,6286707,3.89537,163100055,157115247,5984808,3.66941,65712,NaN
2,01000,AL,Alabama,NaN,NaN,NaN,2133223,2035594,97629,4.6,...,2216627,2130845,85782,3.9,2241747,2174483,67264,3,51771,100
3,01001,AL,"Autauga County, AL",2,2,1,21720,20846,874,4,...,26196,25261,935,3.6,26172,25458,714,2.7,58233,112.482
4,01003,AL,"Baldwin County, AL",3,2,1,69533,66971,2562,3.7,...,95233,91809,3424,3.6,97328,94675,2653,2.7,59871,115.646
5,01005,AL,"Barbour County, AL",6,6,0,11373,10748,625,5.5,...,8414,7987,427,5.1,8537,8213,324,3.8,35972,69.4829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271,72145,PR,"Vega Baja Municipio, PR",1,1,1,19805,17607,2198,11.1,...,13117,11750,1367,10.4,13037,11791,1246,9.6,NaN,NaN
3272,72147,PR,"Vieques Municipio, PR",7,12,0,2441,2120,321,13.2,...,2718,2269,449,16.5,2585,2406,179,6.9,NaN,NaN
3273,72149,PR,"Villalba Municipio, PR",2,2,1,7636,6648,988,12.9,...,7311,6154,1157,15.8,7406,6231,1175,15.9,NaN,NaN
3274,72151,PR,"Yabucoa Municipio, PR",1,1,1,10613,9005,1608,15.2,...,8762,7509,1253,14.3,8691,7552,1139,13.1,NaN,NaN


* fix header columns of csv file
* remove empty rows

In [206]:
# used to remove other year data to keep only 2017-2019
display(unemployment2017_2019.columns)
cols_to_remove = unemployment2017_2019.iloc[:, 3:74].columns
unemployment2017_2019 = remove_col(unemployment2017_2019, cols_to_remove)

unemployment_rates_2017_2019 = unemployment2017_2019.iloc[:, [0, 1,2,6, 10, 14]]
display(unemployment_rates_2017_2019)


Index(['fips_txt', 'Stabr', 'area_name', 'Rural_urban_continuum_code_2013',
       'Urban_influence_code_2013', 'Metro_2013', 'Civilian_labor_force_2000',
       'Employed_2000', 'Unemployed_2000', 'Unemployment_rate_2000',
       'Civilian_labor_force_2001', 'Employed_2001', 'Unemployed_2001',
       'Unemployment_rate_2001', 'Civilian_labor_force_2002', 'Employed_2002',
       'Unemployed_2002', 'Unemployment_rate_2002',
       'Civilian_labor_force_2003', 'Employed_2003', 'Unemployed_2003',
       'Unemployment_rate_2003', 'Civilian_labor_force_2004', 'Employed_2004',
       'Unemployed_2004', 'Unemployment_rate_2004',
       'Civilian_labor_force_2005', 'Employed_2005', 'Unemployed_2005',
       'Unemployment_rate_2005', 'Civilian_labor_force_2006', 'Employed_2006',
       'Unemployed_2006', 'Unemployment_rate_2006',
       'Civilian_labor_force_2007', 'Employed_2007', 'Unemployed_2007',
       'Unemployment_rate_2007', 'Civilian_labor_force_2008', 'Employed_2008',
       'Unemploy

,fips_txt,Stabr,area_name,Unemployment_rate_2017,Unemployment_rate_2018,Unemployment_rate_2019
1,00000,US,United States,4.35512,3.89537,3.66941
2,01000,AL,Alabama,4.4,3.9,3
3,01001,AL,"Autauga County, AL",3.9,3.6,2.7
4,01003,AL,"Baldwin County, AL",4.1,3.6,2.7
5,01005,AL,"Barbour County, AL",5.8,5.1,3.8
...,...,...,...,...,...,...
3271,72145,PR,"Vega Baja Municipio, PR",12.4,10.4,9.6
3272,72147,PR,"Vieques Municipio, PR",14.5,16.5,6.9
3273,72149,PR,"Villalba Municipio, PR",19.4,15.8,15.9
3274,72151,PR,"Yabucoa Municipio, PR",16.5,14.3,13.1


* remove all other column data for years 2000 - 2016
* make a df for only unemployment rates

In [207]:
# Retain only counties and not regions / individual state

UE_counties = unemployment_rates_2017_2019['area_name'].str.contains(',')
UE_county_rate = unemployment_rates_2017_2019.loc[UE_counties]

In [208]:
# check for missing values and dupes

display(UE_county_rate.isnull().sum())
display(UE_county_rate.loc[UE_county_rate.isnull().any(axis=1)])

daily_aqi_by_county_2018.loc[daily_aqi_by_county_2018['State Name'].str.contains('Alaska') & daily_aqi_by_county_2018['county Name'].str.contains('Prince')]
daily_aqi_by_county_2018.loc[daily_aqi_by_county_2018['State Name'].str.contains('Alaska') & daily_aqi_by_county_2018['county Name'].str.contains('Skagway')]
daily_aqi_by_county_2018.loc[daily_aqi_by_county_2018['State Name'].str.contains('Alaska') & daily_aqi_by_county_2018['county Name'].str.contains('Wrangell')]

UE_county_rate.dropna(inplace=True)
UE_county_rate.reset_index(drop=True, inplace=True)

UE_county_rate.rename(columns={'fips_txt':'GeoFIPS'}, inplace=True)

fips_txt                  0
Stabr                     0
area_name                 0
Unemployment_rate_2017    3
Unemployment_rate_2018    3
Unemployment_rate_2019    3
dtype: int64

,fips_txt,Stabr,area_name,Unemployment_rate_2017,Unemployment_rate_2018,Unemployment_rate_2019
93,02201,AK,"Prince of Wales-Outer Ketchikan Census Area, AK",NaN,NaN,NaN
96,02232,AK,"Skagway-Hoonah-Angoon Census Area, AK",NaN,NaN,NaN
100,02280,AK,"Wrangell-Petersburg Census Area, AK",NaN,NaN,NaN


<ipython-input-208-13c24cce17c7>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UE_county_rate.dropna(inplace=True)
/home/wkc/.local/lib/python3.8/site-packages/pandas/core/frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


* 3 alaskan counties show unknown values;
* These values will not be mergeable with aqi data as it we do not aqi data on those areas
* drop these rows
* Rename fips_txt to GeoFIPS

In [209]:
# check if have any duplicate value
print(UE_county_rate['area_name'].duplicated().sum())

0


* we have 0 duplicates

In [210]:
display(UE_county_rate.info())

UE_county_rate['Unemployment_rate_2017'] = pd.to_numeric(UE_county_rate['Unemployment_rate_2017'], downcast='float')
UE_county_rate['Unemployment_rate_2018'] = pd.to_numeric(UE_county_rate['Unemployment_rate_2018'], downcast='float')
UE_county_rate['Unemployment_rate_2019'] = pd.to_numeric(UE_county_rate['Unemployment_rate_2019'], downcast='float')

UE_county_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3218 entries, 0 to 3217
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   GeoFIPS                 3218 non-null   object
 1   Stabr                   3218 non-null   object
 2   area_name               3218 non-null   object
 3   Unemployment_rate_2017  3218 non-null   object
 4   Unemployment_rate_2018  3218 non-null   object
 5   Unemployment_rate_2019  3218 non-null   object
dtypes: object(6)
memory usage: 151.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3218 entries, 0 to 3217
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   GeoFIPS                 3218 non-null   object 
 1   Stabr                   3218 non-null   object 
 2   area_name               3218 non-null   object 
 3   Unemployment_rate_2017  3218 non-null   float32
 4   Unemplo

None

<ipython-input-210-0645ef7acb33>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UE_county_rate['Unemployment_rate_2017'] = pd.to_numeric(UE_county_rate['Unemployment_rate_2017'], downcast='float')
<ipython-input-210-0645ef7acb33>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UE_county_rate['Unemployment_rate_2018'] = pd.to_numeric(UE_county_rate['Unemployment_rate_2018'], downcast='float')
<ipython-input-210-0645ef7acb33>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

* convert Unemployment rate columsn to be type float


In [211]:
#order then plot top 100 and bottom 100
UE_county_rate
display(UE_county_rate.sort_values(by='Unemployment_rate_2017').head(10))
display(UE_county_rate.sort_values(by='Unemployment_rate_2018').head(10))
display(UE_county_rate.sort_values(by='Unemployment_rate_2019').head(10))

display(UE_county_rate.sort_values(by='Unemployment_rate_2017', ascending=False).head(10))
display(UE_county_rate.sort_values(by='Unemployment_rate_2018', ascending=False).head(10))
display(UE_county_rate.sort_values(by='Unemployment_rate_2019', ascending=False).head(10))

,GeoFIPS,Stabr,area_name,Unemployment_rate_2017,Unemployment_rate_2018,Unemployment_rate_2019
292,08095,CO,"Phillips County, CO",1.6,1.9,1.6
1999,38023,ND,"Divide County, ND",1.6,1.3,1.4
307,08125,CO,"Yuma County, CO",1.6,1.9,1.6
248,08009,CO,"Baca County, CO",1.6,1.8,1.7
2033,38091,ND,"Steele County, ND",1.7,2.0,2.1
276,08063,CO,"Kit Carson County, CO",1.7,2.0,1.7
275,08061,CO,"Kiowa County, CO",1.7,1.8,1.6
921,20071,KS,"Greeley County, KS",1.8,2.0,2.3
273,08057,CO,"Jackson County, CO",1.8,2.6,2.3
1998,38021,ND,"Dickey County, ND",1.8,1.8,1.9


,GeoFIPS,Stabr,area_name,Unemployment_rate_2017,Unemployment_rate_2018,Unemployment_rate_2019
1999,38023,ND,"Divide County, ND",1.6,1.3,1.4
2671,48301,TX,"Loving County, TX",5.0,1.5,0.7
846,19119,IA,"Lyon County, IA",1.8,1.6,1.6
2000,38025,ND,"Dunn County, ND",2.2,1.6,1.6
2018,38061,ND,"Mountrail County, ND",2.1,1.6,1.4
2014,38053,ND,"McKenzie County, ND",2.5,1.7,1.6
871,19169,IA,"Story County, IA",2.0,1.7,1.9
275,08061,CO,"Kiowa County, CO",1.7,1.8,1.6
2676,48311,TX,"McMullen County, TX",1.9,1.8,1.6
870,19167,IA,"Sioux County, IA",2.0,1.8,1.9


,GeoFIPS,Stabr,area_name,Unemployment_rate_2017,Unemployment_rate_2018,Unemployment_rate_2019
2671,48301,TX,"Loving County, TX",5.0,1.5,0.7
2018,38061,ND,"Mountrail County, ND",2.1,1.6,1.4
1999,38023,ND,"Divide County, ND",1.6,1.3,1.4
2676,48311,TX,"McMullen County, TX",1.9,1.8,1.6
307,08125,CO,"Yuma County, CO",1.6,1.9,1.6
253,08017,CO,"Cheyenne County, CO",1.8,1.8,1.6
292,08095,CO,"Phillips County, CO",1.6,1.9,1.6
2000,38025,ND,"Dunn County, ND",2.2,1.6,1.6
2014,38053,ND,"McKenzie County, ND",2.5,1.7,1.6
2623,48205,TX,"Hartley County, TX",1.9,1.8,1.6


,GeoFIPS,Stabr,area_name,Unemployment_rate_2017,Unemployment_rate_2018,Unemployment_rate_2019
3182,72083,PR,"Las Marias Municipio, PR",20.600000,14.100000,11.600000
3187,72093,PR,"Maricao Municipio, PR",20.100000,15.100000,13.700000
3195,72109,PR,"Patillas Municipio, PR",19.700001,17.500000,15.700000
81,02158,AK,"Kusilvak Census Area, AK",19.600000,19.600000,19.299999
198,06025,CA,"Imperial County, CA",19.500000,18.900000,18.299999
3215,72149,PR,"Villalba Municipio, PR",19.400000,15.800000,15.900000
3180,72079,PR,"Lajas Municipio, PR",19.400000,15.300000,12.800000
3202,72123,PR,"Salinas Municipio, PR",18.700001,14.800000,14.500000
3181,72081,PR,"Lares Municipio, PR",18.600000,16.200001,14.700000
3168,72055,PR,"Guanica Municipio, PR",18.299999,15.500000,14.800000


,GeoFIPS,Stabr,area_name,Unemployment_rate_2017,Unemployment_rate_2018,Unemployment_rate_2019
81,02158,AK,"Kusilvak Census Area, AK",19.600000,19.600000,19.299999
198,06025,CA,"Imperial County, CA",19.500000,18.900000,18.299999
3195,72109,PR,"Patillas Municipio, PR",19.700001,17.500000,15.700000
110,04027,AZ,"Yuma County, AZ",16.900000,16.700001,16.400000
3214,72147,PR,"Vieques Municipio, PR",14.500000,16.500000,6.900000
3181,72081,PR,"Lares Municipio, PR",18.600000,16.200001,14.700000
3215,72149,PR,"Villalba Municipio, PR",19.400000,15.800000,15.900000
3168,72055,PR,"Guanica Municipio, PR",18.299999,15.500000,14.800000
3180,72079,PR,"Lajas Municipio, PR",19.400000,15.300000,12.800000
3188,72095,PR,"Maunabo Municipio, PR",17.299999,15.200000,15.100000


,GeoFIPS,Stabr,area_name,Unemployment_rate_2017,Unemployment_rate_2018,Unemployment_rate_2019
81,02158,AK,"Kusilvak Census Area, AK",19.600000,19.600000,19.299999
198,06025,CA,"Imperial County, CA",19.500000,18.900000,18.299999
110,04027,AZ,"Yuma County, AZ",16.900000,16.700001,16.400000
3215,72149,PR,"Villalba Municipio, PR",19.400000,15.800000,15.900000
3195,72109,PR,"Patillas Municipio, PR",19.700001,17.500000,15.700000
1430,28063,MS,"Jefferson County, MS",14.600000,13.200000,15.500000
3140,72001,PR,"Adjuntas Municipio, PR",15.100000,13.700000,15.300000
3188,72095,PR,"Maunabo Municipio, PR",17.299999,15.200000,15.100000
3168,72055,PR,"Guanica Municipio, PR",18.299999,15.500000,14.800000
3181,72081,PR,"Lares Municipio, PR",18.600000,16.200001,14.700000


* Don't have consistent low unemployment by years
* Puerto Rico counties appear to have highest unemployment rate over time



* In 2017, it appears that Unemployment rates from 2.5 to 7.5 are normal
* data is skewed to right



# clean up saipe data 2017


In [212]:

saipe_header = saipe2017.iloc[2, :].values
saipe_header
saipe2017.columns = saipe_header
saipe2017.drop([0,1,2], inplace=True)
saipe2017.reset_index(drop=True, inplace=True)
display(saipe2017)

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
0,00,000,US,United States,42583651,42342619,42824683,13.4,13.3,13.5,...,17.5,60336,60250,60422,3932969,3880645,3985293,20.2,19.9,20.5
1,01,000,AL,Alabama,802263,784517,820009,16.9,16.5,17.3,...,23.8,48193,47451,48935,78986,75009,82963,27.7,26.3,29.1
2,01,001,AL,Autauga County,7390,6147,8633,13.4,11.1,15.7,...,22.1,58343,52121,64565,.,.,.,.,.,.
3,01,003,AL,Baldwin County,21199,17444,24954,10.1,8.3,11.9,...,17.9,56607,52439,60775,.,.,.,.,.,.
4,01,005,AL,Barbour County,7414,6325,8503,33.4,28.5,38.3,...,55,32490,29218,35762,.,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,56,037,WY,Sweetwater County,4253,3540,4966,10,8.3,11.7,...,14,75590,69190,81990,.,.,.,.,.,.
3190,56,039,WY,Teton County,1348,1021,1675,5.8,4.4,7.2,...,7.9,90145,80434,99856,.,.,.,.,.,.
3191,56,041,WY,Uinta County,2067,1623,2511,10.2,8,12.4,...,13.4,67404,60925,73883,.,.,.,.,.,.
3192,56,043,WY,Washakie County,845,632,1058,10.7,8,13.4,...,17.3,57989,53142,62836,.,.,.,.,.,.


In [213]:
saipe2017.columns
saipe2017.iloc[:, [1,2,3,4,7]]
saipe_county = saipe2017['Name'].str.contains('County')
pov_rate_2017 = saipe2017.loc[saipe_county].iloc[:, [0, 1,2,3, 7]]
pov_rate_2017.reset_index(drop=True, inplace=True)
display(pov_rate_2017)

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Percent, All Ages"
0,01,001,AL,Autauga County,13.4
1,01,003,AL,Baldwin County,10.1
2,01,005,AL,Barbour County,33.4
3,01,007,AL,Bibb County,20.2
4,01,009,AL,Blount County,12.8
...,...,...,...,...,...
3002,56,037,WY,Sweetwater County,10
3003,56,039,WY,Teton County,5.8
3004,56,041,WY,Uinta County,10.2
3005,56,043,WY,Washakie County,10.7


* Remove empty cells and clean up columns
* Retain only US county information and poverty rates for all ages


In [214]:
print(pov_rate_2017.isnull().sum())
print(pov_rate_2017.duplicated().sum())
display(pov_rate_2017.info())

State FIPS Code              0
County FIPS Code             0
Postal Code                  0
Name                         0
Poverty Percent, All Ages    0
dtype: int64
0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3007 entries, 0 to 3006
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   State FIPS Code            3007 non-null   object
 1   County FIPS Code           3007 non-null   object
 2   Postal Code                3007 non-null   object
 3   Name                       3007 non-null   object
 4   Poverty Percent, All Ages  3007 non-null   object
dtypes: object(5)
memory usage: 117.6+ KB


None

* No duplicates or missing values for poverty rates in 2017
* also convert poverty percent to float

In [215]:
pov_rate_2017.info()

pov_rate_2017.drop(518, inplace=True)
pov_rate_2017['Poverty Percent, All Ages'] = pd.to_numeric(pov_rate_2017['Poverty Percent, All Ages'], downcast='float')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3007 entries, 0 to 3006
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   State FIPS Code            3007 non-null   object
 1   County FIPS Code           3007 non-null   object
 2   Postal Code                3007 non-null   object
 3   Name                       3007 non-null   object
 4   Poverty Percent, All Ages  3007 non-null   object
dtypes: object(5)
memory usage: 117.6+ KB


* from further inspection it was noted that Kalawao county is now excluded in their survey results starting 2017
* Kalawao county records removed
* Poverty rates in Hawaii appear to around 10-20s

In [216]:
display(pov_rate_2017.sort_values(by='Poverty Percent, All Ages').head(10))
display(pov_rate_2017.sort_values(by='Poverty Percent, All Ages', ascending=False).head(10))

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Percent, All Ages"
2775,51,107,VA,Loudoun County,3.0
233,08,035,CO,Douglas County,3.2
2305,46,083,SD,Lincoln County,3.5
1713,35,028,NM,Los Alamos County,3.7
695,18,057,IN,Hamilton County,3.8
1686,34,019,NJ,Hunterdon County,3.9
2424,47,187,TN,Williamson County,3.9
1288,27,139,MN,Scott County,4.0
1228,27,019,MN,Carver County,4.0
2694,49,029,UT,Morgan County,4.1


,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Percent, All Ages"
2330,46,137,SD,Ziebach County,56.700001
2324,46,121,SD,Todd County,50.400002
228,08,025,CO,Crowley County,47.400002
2272,46,017,SD,Buffalo County,43.299999
1316,28,021,MS,Claiborne County,42.599998
2279,46,031,SD,Corson County,42.400002
1347,28,083,MS,Leflore County,42.200001
988,21,051,KY,Clay County,41.700001
2315,46,102,SD,Oglala Lakota County,41.500000
1010,21,095,KY,Harlan County,41.500000


* Puerto Rico is not found and that is because poverty county data for Puerto Rico is in a separate set
* Need to add Puerto Rico

# Repeat clean up steps on saipe data 2018


In [217]:
# clean up saipe data 2018

saipe_header = saipe2018.iloc[2, :].values
saipe_header
saipe2018.columns = saipe_header
saipe2018.drop([0,1,2], inplace=True)
saipe2018.reset_index(drop=True, inplace=True)
saipe2018

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
0,00,000,US,United States,41852315,41619366,42085264,13.1,13,13.2,...,17.2,61937,61843,62031,3758704,3714862,3802546,19.5,19.3,19.7
1,01,000,AL,Alabama,801758,785668,817848,16.8,16.5,17.1,...,23.7,49881,49123,50639,73915,69990,77840,26,24.6,27.4
2,01,001,AL,Autauga County,7587,6334,8840,13.8,11.5,16.1,...,23.9,59338,53628,65048,.,.,.,.,.,.
3,01,003,AL,Baldwin County,21069,17390,24748,9.8,8.1,11.5,...,16.9,57588,54437,60739,.,.,.,.,.,.
4,01,005,AL,Barbour County,6788,5662,7914,30.9,25.8,36,...,45.9,34382,31157,37607,.,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,56,037,WY,Sweetwater County,3540,2804,4276,8.4,6.7,10.1,...,11.2,73315,67141,79489,.,.,.,.,.,.
3190,56,039,WY,Teton County,1443,1137,1749,6.3,5,7.6,...,7.7,99087,88142,110032,.,.,.,.,.,.
3191,56,041,WY,Uinta County,2010,1585,2435,10,7.9,12.1,...,14,63401,56812,69990,.,.,.,.,.,.
3192,56,043,WY,Washakie County,918,700,1136,11.9,9.1,14.7,...,19.3,55190,49263,61117,.,.,.,.,.,.


In [218]:
saipe2018.columns
saipe2018.iloc[:, [1,2,3,4,7]]
saipe_county = saipe2018['Name'].str.contains('County')
pov_rate_2018 = saipe2018.loc[saipe_county].iloc[:, [0, 1,2,3, 7]]
pov_rate_2018.reset_index(drop=True, inplace=True)
display(pov_rate_2018)

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Percent, All Ages"
0,01,001,AL,Autauga County,13.8
1,01,003,AL,Baldwin County,9.8
2,01,005,AL,Barbour County,30.9
3,01,007,AL,Bibb County,21.8
4,01,009,AL,Blount County,13.2
...,...,...,...,...,...
3002,56,037,WY,Sweetwater County,8.4
3003,56,039,WY,Teton County,6.3
3004,56,041,WY,Uinta County,10
3005,56,043,WY,Washakie County,11.9


* Remove empty cells and clean up columns
* Retain only US county information and poverty rates for all ages


In [219]:
print(pov_rate_2018.isnull().sum())
print(pov_rate_2018.duplicated().sum())
display(pov_rate_2018.info())

State FIPS Code              0
County FIPS Code             0
Postal Code                  0
Name                         0
Poverty Percent, All Ages    0
dtype: int64
0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3007 entries, 0 to 3006
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   State FIPS Code            3007 non-null   object
 1   County FIPS Code           3007 non-null   object
 2   Postal Code                3007 non-null   object
 3   Name                       3007 non-null   object
 4   Poverty Percent, All Ages  3007 non-null   object
dtypes: object(5)
memory usage: 117.6+ KB


None

* No duplicates or missing values for poverty rates in 2018
* also convert poverty percent to float

In [220]:
pov_rate_2018.drop(518, inplace=True)
pov_rate_2018['Poverty Percent, All Ages'] = pd.to_numeric(pov_rate_2018['Poverty Percent, All Ages'], downcast='float')


* from further inspection it was noted that Kalawao county is now excluded in their survey results starting 2017
* Kalawao county records removed
* Poverty rates in Hawaii appear to around 10-20s

In [221]:
display(pov_rate_2018.sort_values(by='Poverty Percent, All Ages').head(10))
display(pov_rate_2018.sort_values(by='Poverty Percent, All Ages', ascending=False).head(10))

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Percent, All Ages"
233,08,035,CO,Douglas County,2.6
2576,48,301,TX,Loving County,3.3
2775,51,107,VA,Loudoun County,3.6
2424,47,187,TN,Williamson County,3.8
1228,27,019,MN,Carver County,3.8
1713,35,028,NM,Los Alamos County,3.9
2694,49,029,UT,Morgan County,4.0
1966,39,041,OH,Delaware County,4.1
611,17,093,IL,Kendall County,4.2
2957,55,089,WI,Ozaukee County,4.2


,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Percent, All Ages"
2315,46,102,SD,Oglala Lakota County,54.000000
2324,46,121,SD,Todd County,48.400002
2272,46,017,SD,Buffalo County,45.700001
228,08,025,CO,Crowley County,44.299999
2330,46,137,SD,Ziebach County,43.900002
120,05,077,AR,Lee County,43.000000
5,01,011,AL,Bullock County,42.500000
1333,28,055,MS,Issaquena County,40.500000
509,13,309,GA,Wheeler County,39.599998
1057,21,189,KY,Owsley County,39.200001


* Puerto Rico is not found and that is because poverty county data for Puerto Rico is in a separate set
* But poverty ranking appear to be same from 2017

# Repeat clean up steps for saipe data 2019

In [222]:

saipe_header = saipe2019.iloc[2, :].values
saipe_header
saipe2019.columns = saipe_header
saipe2019.drop([0,1,2], inplace=True)
saipe2019.reset_index(drop=True, inplace=True)
saipe2019

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
0,00,000,US,United States,39490096,39248096,39732096,12.3,12.2,12.4,...,16,65712,65594,65830,3457689,3405854,3509524,18.2,17.9,18.5
1,01,000,AL,Alabama,747478,730491,764465,15.6,15.2,16,...,21.6,51771,51179,52363,69236,65296,73176,24.2,22.8,25.6
2,01,001,AL,Autauga County,6723,5517,7929,12.1,9.9,14.3,...,19.4,58233,52517,63949,.,.,.,.,.,.
3,01,003,AL,Baldwin County,22360,18541,26179,10.1,8.4,11.8,...,17.2,59871,54593,65149,.,.,.,.,.,.
4,01,005,AL,Barbour County,5909,4787,7031,27.1,22,32.2,...,49,35972,31822,40122,.,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,56,037,WY,Sweetwater County,3453,2743,4163,8.3,6.6,10,...,11.1,80639,73437,87841,.,.,.,.,.,.
3190,56,039,WY,Teton County,1396,1073,1719,6,4.6,7.4,...,6.7,98837,86531,111143,.,.,.,.,.,.
3191,56,041,WY,Uinta County,1699,1264,2134,8.5,6.3,10.7,...,11.1,70756,63191,78321,.,.,.,.,.,.
3192,56,043,WY,Washakie County,845,626,1064,11.1,8.2,14,...,17.4,55122,50050,60194,.,.,.,.,.,.


In [223]:
saipe2019.columns
saipe2019.iloc[:, [1,2,3,4,7]]
saipe_county = saipe2019['Name'].str.contains('County')
pov_rate_2019 = saipe2019.loc[saipe_county].iloc[:, [0, 1,2,3, 7]]
pov_rate_2019.reset_index(drop=True, inplace=True)
display(pov_rate_2019)

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Percent, All Ages"
0,01,001,AL,Autauga County,12.1
1,01,003,AL,Baldwin County,10.1
2,01,005,AL,Barbour County,27.1
3,01,007,AL,Bibb County,20.3
4,01,009,AL,Blount County,16.3
...,...,...,...,...,...
3002,56,037,WY,Sweetwater County,8.3
3003,56,039,WY,Teton County,6
3004,56,041,WY,Uinta County,8.5
3005,56,043,WY,Washakie County,11.1


* Remove empty cells and clean up columns
* Retain only US county information and poverty rates for all ages


In [224]:
print(pov_rate_2019.isnull().sum())
print(pov_rate_2019.duplicated().sum())
display(pov_rate_2019.info())

State FIPS Code              0
County FIPS Code             0
Postal Code                  0
Name                         0
Poverty Percent, All Ages    0
dtype: int64
0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3007 entries, 0 to 3006
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   State FIPS Code            3007 non-null   object
 1   County FIPS Code           3007 non-null   object
 2   Postal Code                3007 non-null   object
 3   Name                       3007 non-null   object
 4   Poverty Percent, All Ages  3007 non-null   object
dtypes: object(5)
memory usage: 117.6+ KB


None

* No duplicates or missing values for poverty rates in 2018
* also convert poverty percent to float

In [225]:
pov_rate_2019.drop(518, inplace=True)

pov_rate_2019['Poverty Percent, All Ages'] = pd.to_numeric(pov_rate_2019['Poverty Percent, All Ages'], downcast='float')


* from further inspection it was noted that Kalawao county is now excluded in their survey results starting 2017
* Kalawao county records removed
* Poverty rates in Hawaii appear to around 10-20s


In [226]:
display(pov_rate_2019.sort_values(by='Poverty Percent, All Ages').head(10))
display(pov_rate_2019.sort_values(by='Poverty Percent, All Ages', ascending=False).head(10))

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Percent, All Ages"
233,08,035,CO,Douglas County,2.7
2775,51,107,VA,Loudoun County,3.1
1713,35,028,NM,Los Alamos County,3.5
2305,46,083,SD,Lincoln County,3.8
1686,34,019,NJ,Hunterdon County,4.0
611,17,093,IL,Kendall County,4.0
2694,49,029,UT,Morgan County,4.0
222,08,014,CO,Broomfield County,4.1
695,18,057,IN,Hamilton County,4.2
1288,27,139,MN,Scott County,4.2


,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Percent, All Ages"
2330,46,137,SD,Ziebach County,47.700001
2324,46,121,SD,Todd County,43.400002
2279,46,031,SD,Corson County,40.299999
2315,46,102,SD,Oglala Lakota County,40.099998
228,08,025,CO,Crowley County,40.000000
2272,46,017,SD,Buffalo County,39.799999
1319,28,027,MS,Coahoma County,38.200001
1373,28,135,MS,Tallahatchie County,37.900002
1316,28,021,MS,Claiborne County,37.500000
1332,28,053,MS,Humphreys County,37.099998


* Puerto Rico is not found and that is because poverty county data for Puerto Rico is in a separate set
* Need to add Puerto Rico
* But poverty ranking appear to be same from 2017-2019

# Clean education and vehicle 2017 dataset

In [227]:
educ_vehicle_2017.shape
educ_vehicle_2017.columns

educ_vehicle_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837 entries, 0 to 836
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        837 non-null    int64  
 1   NAME              837 non-null    object 
 2   state             837 non-null    int64  
 3   county            837 non-null    int64  
 4   POP               837 non-null    int64  
 5   EDU_TOT           826 non-null    float64
 6   LESS_HS_TOT       826 non-null    float64
 7   HS_TOT            826 non-null    float64
 8   COL_OR_ASSOC_TOT  826 non-null    float64
 9   BACH_TOT          826 non-null    float64
 10  GRAD_TOT          826 non-null    float64
 11  VEHICLE_TOT       837 non-null    int64  
 12  WALK_TOT          226 non-null    float64
dtypes: float64(7), int64(5), object(1)
memory usage: 85.1+ KB


* Have 837 rows and 12 columns
* Category types are appropriate for numerical and nominal columns

In [228]:
# educ_vehicle_2017.drop('Unnamed: 0', axis=1, inplace=True)
educ_vehicle_2017['WALK_TOT'].unique()
educ_vehicle_2017['WALK_TOT'].isnull().sum()

#drop walk b/c too many missing values and only retain counties and drop also drop that row index column
educ_vehicle_2017.state.unique()
cols_to_remove = ['Unnamed: 0', 'WALK_TOT']
educ_vehicle_2017 = remove_col(educ_vehicle_2017, cols_to_remove)

* drop first column as it is just an index column
* drop total walk column as majority of data is missing
* nb there is no fips state code for 4

In [229]:
educ_vehicle_2017.NAME.duplicated().sum()
educ_vehicle_2017.isnull().sum()
educ_vehicle_2017.loc[educ_vehicle_2017.isnull().any(axis=1)]
educ_vehicle_2017.loc[educ_vehicle_2017.NAME.str.contains('Puerto')]

,NAME,state,county,POP,EDU_TOT,LESS_HS_TOT,HS_TOT,COL_OR_ASSOC_TOT,BACH_TOT,GRAD_TOT,VEHICLE_TOT
352,"Arecibo Municipio, Puerto Rico",72,13,86066,NaN,NaN,NaN,NaN,NaN,NaN,21075
353,"Bayamón Municipio, Puerto Rico",72,21,179565,NaN,NaN,NaN,NaN,NaN,NaN,46000
354,"Caguas Municipio, Puerto Rico",72,25,129604,NaN,NaN,NaN,NaN,NaN,NaN,36045
355,"Carolina Municipio, Puerto Rico",72,31,154489,NaN,NaN,NaN,NaN,NaN,NaN,52510
356,"Guaynabo Municipio, Puerto Rico",72,61,87328,NaN,NaN,NaN,NaN,NaN,NaN,25965
357,"Mayagüez Municipio, Puerto Rico",72,97,75525,NaN,NaN,NaN,NaN,NaN,NaN,15815
358,"Ponce Municipio, Puerto Rico",72,113,140859,NaN,NaN,NaN,NaN,NaN,NaN,32020
359,"San Juan Municipio, Puerto Rico",72,127,337288,NaN,NaN,NaN,NaN,NaN,NaN,95110
360,"Toa Alta Municipio, Puerto Rico",72,135,73217,NaN,NaN,NaN,NaN,NaN,NaN,21330
361,"Toa Baja Municipio, Puerto Rico",72,137,78092,NaN,NaN,NaN,NaN,NaN,NaN,24670


* All Missing values are Puerto Rico counties similar to previous data sets
* Census Bureau keeps Puerto Rico as separate data column for education
* No duplicated data shown

In [230]:
edu_vehicle_estimates = educ_vehicle_2017.columns.values[4:]
# educ_vehicle_2017

for est in edu_vehicle_estimates:
    educ_vehicle_2017[est + '_ratio'] = educ_vehicle_2017[est] / educ_vehicle_2017['POP'] * 100
    educ_vehicle_2017.drop(est, axis=1, inplace=True)

# convert into ratios
display(educ_vehicle_2017)

,NAME,state,county,POP,EDU_TOT_ratio,LESS_HS_TOT_ratio,HS_TOT_ratio,COL_OR_ASSOC_TOT_ratio,BACH_TOT_ratio,GRAD_TOT_ratio,VEHICLE_TOT_ratio
0,"St. Lawrence County, New York",36,89,109623,66.185928,8.359560,23.588116,18.327358,7.552247,8.358647,32.766846
1,"Saratoga County, New York",36,91,229869,71.525086,4.635684,16.980541,19.820419,16.481561,13.606880,45.984887
2,"Schenectady County, New York",36,93,155565,69.072735,6.482821,20.662745,22.667695,11.062900,8.196574,38.755504
3,"Steuben County, New York",36,101,96281,70.638028,6.761459,26.921199,21.483990,7.738806,7.732574,36.959525
4,"Suffolk County, New York",36,103,1492953,69.456038,6.873693,18.559191,18.702062,13.936407,11.384685,41.375382
...,...,...,...,...,...,...,...,...,...,...,...
832,"Putnam County, New York",36,79,99323,71.933993,5.124694,20.737392,18.218338,15.030758,12.822810,42.683970
833,"Queens County, New York",36,81,2358582,71.878018,12.930820,20.936054,15.809711,14.245424,7.956009,16.619308
834,"Rensselaer County, New York",36,83,159722,69.525175,5.694269,19.865767,21.884274,12.611287,9.469578,41.146492
835,"Richmond County, New York",36,85,479458,69.678470,8.140650,21.009139,17.452832,13.788069,9.287779,26.693683


* convert all estimates of education and vehicles of a county to a ratio of estimated total / population of county
* retrieve ratio estimate to county population


# Repeat Steps for education and vehicle 2018 data

In [231]:
educ_vehicle_2018.shape
educ_vehicle_2018.columns

educ_vehicle_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838 entries, 0 to 837
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        838 non-null    int64  
 1   NAME              838 non-null    object 
 2   state             838 non-null    int64  
 3   county            838 non-null    int64  
 4   POP               838 non-null    int64  
 5   EDU_TOT           826 non-null    float64
 6   LESS_HS_TOT       826 non-null    float64
 7   HS_TOT            826 non-null    float64
 8   COL_OR_ASSOC_TOT  826 non-null    float64
 9   BACH_TOT          826 non-null    float64
 10  GRAD_TOT          826 non-null    float64
 11  VEHICLE_TOT       838 non-null    int64  
 12  WALK_TOT          231 non-null    float64
dtypes: float64(7), int64(5), object(1)
memory usage: 85.2+ KB


* Have 837 rows and 12 columns
* Category types are appropriate for numerical and nominal columns

In [232]:
educ_vehicle_2018['WALK_TOT'].unique()
educ_vehicle_2018['WALK_TOT'].isnull().sum()

educ_vehicle_2018.state.unique()
cols_to_remove = ['Unnamed: 0', 'WALK_TOT']
educ_vehicle_2018 = remove_col(educ_vehicle_2018, cols_to_remove)

* drop first column as it is just an index column
* drop total walk column as majority of data is missing
* nb there is no fips state code for 4

In [233]:
educ_vehicle_2018.NAME.duplicated().sum()
educ_vehicle_2018.isnull().sum()
educ_vehicle_2018.loc[educ_vehicle_2018.isnull().any(axis=1)]

,NAME,state,county,POP,EDU_TOT,LESS_HS_TOT,HS_TOT,COL_OR_ASSOC_TOT,BACH_TOT,GRAD_TOT,VEHICLE_TOT
376,"Jones County, Mississippi",28,67,68461,NaN,NaN,NaN,NaN,NaN,NaN,26755
827,"Arecibo Municipio, Puerto Rico",72,13,82114,NaN,NaN,NaN,NaN,NaN,NaN,21210
828,"Bayamón Municipio, Puerto Rico",72,21,170480,NaN,NaN,NaN,NaN,NaN,NaN,48105
829,"Caguas Municipio, Puerto Rico",72,25,124434,NaN,NaN,NaN,NaN,NaN,NaN,37810
830,"Carolina Municipio, Puerto Rico",72,31,147661,NaN,NaN,NaN,NaN,NaN,NaN,47205
831,"Guaynabo Municipio, Puerto Rico",72,61,83787,NaN,NaN,NaN,NaN,NaN,NaN,27890
832,"Mayagüez Municipio, Puerto Rico",72,97,71986,NaN,NaN,NaN,NaN,NaN,NaN,14200
833,"Ponce Municipio, Puerto Rico",72,113,133191,NaN,NaN,NaN,NaN,NaN,NaN,32475
834,"San Juan Municipio, Puerto Rico",72,127,320967,NaN,NaN,NaN,NaN,NaN,NaN,94245
835,"Toa Alta Municipio, Puerto Rico",72,135,71094,NaN,NaN,NaN,NaN,NaN,NaN,24755


* All Missing values are Puerto Rico counties similar to previous data sets
* Census Bureau keeps Puerto Rico as separate data column for education
* No duplicated data shown

In [234]:
edu_vehicle_estimates = educ_vehicle_2018.columns.values[4:]
# educ_vehicle_2017

for est in edu_vehicle_estimates:
    educ_vehicle_2018[est + '_ratio'] = educ_vehicle_2018[est] / educ_vehicle_2018['POP'] * 100
    educ_vehicle_2018.drop(est, axis=1, inplace=True)

# convert into ratios
display(educ_vehicle_2018)

,NAME,state,county,POP,EDU_TOT_ratio,LESS_HS_TOT_ratio,HS_TOT_ratio,COL_OR_ASSOC_TOT_ratio,BACH_TOT_ratio,GRAD_TOT_ratio,VEHICLE_TOT_ratio
0,"Baldwin County, Alabama",1,3,218022,71.410683,7.009384,20.187412,21.820275,14.773738,7.619873,40.245938
1,"Calhoun County, Alabama",1,15,114277,69.280783,10.606684,22.542594,23.660929,6.925278,5.545298,37.417853
2,"Cullman County, Alabama",1,43,83442,69.891661,12.257616,21.750437,27.423839,5.891517,2.568251,39.626327
3,"DeKalb County, Alabama",1,49,71385,67.477761,16.363382,20.078448,21.005814,5.506759,4.523359,36.632346
4,"Elmore County, Alabama",1,51,81887,69.355331,9.146751,22.539597,20.155824,12.390245,5.122913,38.589764
...,...,...,...,...,...,...,...,...,...,...,...
833,"Ponce Municipio, Puerto Rico",72,113,133191,NaN,NaN,NaN,NaN,NaN,NaN,24.382278
834,"San Juan Municipio, Puerto Rico",72,127,320967,NaN,NaN,NaN,NaN,NaN,NaN,29.362832
835,"Toa Alta Municipio, Puerto Rico",72,135,71094,NaN,NaN,NaN,NaN,NaN,NaN,34.820097
836,"Toa Baja Municipio, Puerto Rico",72,137,74623,NaN,NaN,NaN,NaN,NaN,NaN,31.759645


* convert all estimates of education and vehicles of a county to a ratio of estimated total / population of county
* retrieve ratio estimate to county population


# Repeat Steps for education and vehicle 2019 data

In [235]:
educ_vehicle_2019.shape
educ_vehicle_2019.columns

educ_vehicle_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        840 non-null    int64  
 1   NAME              840 non-null    object 
 2   state             840 non-null    int64  
 3   county            840 non-null    int64  
 4   POP               840 non-null    int64  
 5   EDU_TOT           827 non-null    float64
 6   LESS_HS_TOT       827 non-null    float64
 7   HS_TOT            827 non-null    float64
 8   COL_OR_ASSOC_TOT  827 non-null    float64
 9   BACH_TOT          827 non-null    float64
 10  GRAD_TOT          827 non-null    float64
 11  VEHICLE_TOT       840 non-null    int64  
 12  WALK_TOT          226 non-null    float64
dtypes: float64(7), int64(5), object(1)
memory usage: 85.4+ KB


* Have 837 rows and 12 columns
* Category types are appropriate for numerical and nominal columns

In [236]:
educ_vehicle_2019['WALK_TOT'].unique()
educ_vehicle_2019['WALK_TOT'].isnull().sum()

educ_vehicle_2019.state.unique()
cols_to_remove = ['Unnamed: 0', 'WALK_TOT']
educ_vehicle_2019 = remove_col(educ_vehicle_2019, cols_to_remove)

* drop first column as it is just an index column
* drop total walk column as majority of data is missing
* nb there is no fips state code for 4

In [237]:
educ_vehicle_2019.NAME.duplicated().sum()
educ_vehicle_2019.isnull().sum()
educ_vehicle_2019.loc[educ_vehicle_2019.isnull().any(axis=1)]

,NAME,state,county,POP,EDU_TOT,LESS_HS_TOT,HS_TOT,COL_OR_ASSOC_TOT,BACH_TOT,GRAD_TOT,VEHICLE_TOT
291,"St. Landry Parish, Louisiana",22,97,82124,NaN,NaN,NaN,NaN,NaN,NaN,27175
559,"Muskingum County, Ohio",39,119,86215,NaN,NaN,NaN,NaN,NaN,NaN,33710
829,"Arecibo Municipio, Puerto Rico",72,13,81966,NaN,NaN,NaN,NaN,NaN,NaN,20780
830,"Bayamón Municipio, Puerto Rico",72,21,169269,NaN,NaN,NaN,NaN,NaN,NaN,53730
831,"Caguas Municipio, Puerto Rico",72,25,124606,NaN,NaN,NaN,NaN,NaN,NaN,39305
832,"Carolina Municipio, Puerto Rico",72,31,146984,NaN,NaN,NaN,NaN,NaN,NaN,48385
833,"Guaynabo Municipio, Puerto Rico",72,61,83728,NaN,NaN,NaN,NaN,NaN,NaN,30465
834,"Mayagüez Municipio, Puerto Rico",72,97,71530,NaN,NaN,NaN,NaN,NaN,NaN,16695
835,"Ponce Municipio, Puerto Rico",72,113,131881,NaN,NaN,NaN,NaN,NaN,NaN,31635
836,"San Juan Municipio, Puerto Rico",72,127,318441,NaN,NaN,NaN,NaN,NaN,NaN,92055


* All Missing values are Puerto Rico counties similar to previous data sets
* Census Bureau keeps Puerto Rico as separate data column for education
* No duplicated data shown

In [238]:
edu_vehicle_estimates = educ_vehicle_2019.columns.values[4:]
# educ_vehicle_2017

for est in edu_vehicle_estimates:
    educ_vehicle_2019[est + '_ratio'] = educ_vehicle_2019[est] / educ_vehicle_2019['POP'] * 100
    educ_vehicle_2019.drop(est, axis=1, inplace=True)

# convert into ratios
display(educ_vehicle_2019)

,NAME,state,county,POP,EDU_TOT_ratio,LESS_HS_TOT_ratio,HS_TOT_ratio,COL_OR_ASSOC_TOT_ratio,BACH_TOT_ratio,GRAD_TOT_ratio,VEHICLE_TOT_ratio
0,"Baldwin County, Alabama",1,3,223234,71.546897,6.790184,18.762375,22.937366,14.684591,8.372381,38.152790
1,"Calhoun County, Alabama",1,15,113605,69.613133,10.734563,24.010387,21.438317,7.994366,5.435500,36.380441
2,"Cullman County, Alabama",1,43,83768,70.187900,13.148219,21.397192,24.610830,7.085044,3.946614,38.009741
3,"DeKalb County, Alabama",1,49,71513,65.732105,14.043600,25.003845,18.295974,5.164096,3.224589,39.244613
4,"Elmore County, Alabama",1,51,81209,70.870224,7.314460,24.617961,21.316603,11.597237,6.023963,40.642047
...,...,...,...,...,...,...,...,...,...,...,...
835,"Ponce Municipio, Puerto Rico",72,113,131881,NaN,NaN,NaN,NaN,NaN,NaN,23.987534
836,"San Juan Municipio, Puerto Rico",72,127,318441,NaN,NaN,NaN,NaN,NaN,NaN,28.908024
837,"Toa Alta Municipio, Puerto Rico",72,135,72025,NaN,NaN,NaN,NaN,NaN,NaN,33.641097
838,"Toa Baja Municipio, Puerto Rico",72,137,74271,NaN,NaN,NaN,NaN,NaN,NaN,30.833030


* convert all estimates of education and vehicles of a county to a ratio of estimated total / population of county
* retrieve ratio estimate to county population


# clean AQI and County demographic data

In [239]:
# add GeoFIPS for NE_aqi_2019

aqi_2017 = get_df_with_geofips(daily_aqi_by_county_2017, 'State Code', 'County Code')
aqi_2018 = get_df_with_geofips(daily_aqi_by_county_2018, 'State Code', 'County Code')
aqi_2019 = get_df_with_geofips(daily_aqi_by_county_2019, 'State Code', 'County Code')

In [240]:
## clean AQI data
cols_to_remove = ['Defining Site', 'Number of Sites Reporting', 'State Code', 'County Code']


aqi_2017 = remove_col(aqi_2017, cols_to_remove)
aqi_2018 = remove_col(aqi_2018, cols_to_remove)
aqi_2019 = remove_col(aqi_2019, cols_to_remove)

In [241]:
AQI_metrics = aqi_2017.columns[3:]

for metric in AQI_metrics:
    aqi_2017.rename(columns={metric:metric + '_2017'}, inplace=True)
    aqi_2018.rename(columns={metric:metric + '_2018'}, inplace=True)
    aqi_2019.rename(columns={metric:metric + '_2019'}, inplace=True)


* Give appropriate year name to columns to distinguish them before merging

In [242]:
print(aqi_2017.columns)
aqi_2017.head()


Index(['GeoFIPS', 'State Name', 'county Name', 'Date_2017', 'AQI_2017',
       'Category_2017', 'Defining Parameter_2017'],
      dtype='object')


,GeoFIPS,State Name,county Name,Date_2017,AQI_2017,Category_2017,Defining Parameter_2017
0,01003,Alabama,Baldwin,2017-01-01,21,Good,PM2.5
1,01003,Alabama,Baldwin,2017-01-04,22,Good,PM2.5
2,01003,Alabama,Baldwin,2017-01-10,19,Good,PM2.5
3,01003,Alabama,Baldwin,2017-01-13,30,Good,PM2.5
4,01003,Alabama,Baldwin,2017-01-16,16,Good,PM2.5


In [243]:
print(aqi_2017.columns)
average_aqi_2017 = aqi_2017.groupby(['GeoFIPS', 'State Name', 'county Name'], as_index=False)[['AQI_2017']].mean()
average_aqi_2018 = aqi_2018.groupby(['GeoFIPS', 'State Name', 'county Name'], as_index=False)[['AQI_2018']].mean()
average_aqi_2019 = aqi_2019.groupby(['GeoFIPS', 'State Name', 'county Name'], as_index=False)[['AQI_2019']].mean()

display(average_aqi_2017)
display(average_aqi_2018)
display(average_aqi_2019)

Index(['GeoFIPS', 'State Name', 'county Name', 'Date_2017', 'AQI_2017',
       'Category_2017', 'Defining Parameter_2017'],
      dtype='object')


,GeoFIPS,State Name,county Name,AQI_2017
0,01003,Alabama,Baldwin,37.503704
1,01027,Alabama,Clay,32.457627
2,01033,Alabama,Colbert,36.155477
3,01049,Alabama,DeKalb,38.239554
4,01051,Alabama,Elmore,34.371681
...,...,...,...,...
1057,72127,Puerto Rico,San Juan,9.357724
1058,78010,Virgin Islands,St Croix,24.346154
1059,78020,Virgin Islands,St John,26.223881
1060,80002,Country Of Mexico,BAJA CALIFORNIA NORTE,74.343490


,GeoFIPS,State Name,county Name,AQI_2018
0,01003,Alabama,Baldwin,36.829630
1,01027,Alabama,Clay,28.990909
2,01033,Alabama,Colbert,35.635379
3,01049,Alabama,DeKalb,36.720000
4,01051,Alabama,Elmore,35.328829
...,...,...,...,...
1051,78010,Virgin Islands,St Croix,29.090909
1052,78020,Virgin Islands,St John,21.000000
1053,78030,Virgin Islands,St Thomas,33.207650
1054,80002,Country Of Mexico,BAJA CALIFORNIA NORTE,85.000000


,GeoFIPS,State Name,county Name,AQI_2019
0,01003,Alabama,Baldwin,38.291513
1,01027,Alabama,Clay,31.149533
2,01033,Alabama,Colbert,35.520913
3,01049,Alabama,DeKalb,39.711911
4,01051,Alabama,Elmore,38.421053
...,...,...,...,...
1048,78010,Virgin Islands,St Croix,24.877193
1049,78020,Virgin Islands,St John,21.752475
1050,78030,Virgin Islands,St Thomas,32.738806
1051,80002,Country Of Mexico,BAJA CALIFORNIA NORTE,77.125000


## clean demographic data


In [244]:
county_demographic = county_demographic.loc[county_demographic.YEAR > 9]
# keep only totals of dem; disregard ages
county_demographic = county_demographic.loc[county_demographic.AGEGRP == 0]

* county_demographic contains all county demographic data for US
* Only 2017-2019 data is retained
* All age groups are considered for ethnicities

In [245]:
demographic = county_demographic.columns.values[8:]
county_dem_ratio = county_demographic

for dem in demographic:
    county_dem_ratio[dem] = county_demographic[dem] / county_demographic['TOT_POP'] * 100
    county_dem_ratio.rename(columns={dem:dem + '_ratio'}, inplace=True)

display(county_dem_ratio)

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE_ratio,TOT_FEMALE_ratio,...,HWAC_MALE_ratio,HWAC_FEMALE_ratio,HBAC_MALE_ratio,HBAC_FEMALE_ratio,HIAC_MALE_ratio,HIAC_FEMALE_ratio,HAAC_MALE_ratio,HAAC_FEMALE_ratio,HNAC_MALE_ratio,HNAC_FEMALE_ratio
171,50,1,1,Alabama,Autauga County,10,0,55390,48.763315,51.236685,...,1.261961,1.160859,0.200397,0.140820,0.070410,0.074021,0.021665,0.027081,0.030691,0.027081
190,50,1,1,Alabama,Autauga County,11,0,55533,48.630544,51.369456,...,1.375759,1.210091,0.162066,0.135055,0.070229,0.057623,0.036015,0.028812,0.034214,0.023410
209,50,1,1,Alabama,Autauga County,12,0,55869,48.492008,51.507992,...,1.392543,1.229662,0.159301,0.166461,0.071596,0.048327,0.026849,0.034008,0.028638,0.019689
399,50,1,3,Alabama,Baldwin County,10,0,212521,48.568377,51.431623,...,2.243072,2.000743,0.121870,0.127046,0.112930,0.076228,0.019763,0.026350,0.020704,0.018822
418,50,1,3,Alabama,Baldwin County,11,0,217855,48.519428,51.480572,...,2.281334,2.053201,0.120722,0.120722,0.118427,0.085378,0.028459,0.025705,0.022951,0.014230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716110,50,56,43,Wyoming,Washakie County,11,0,7877,50.577631,49.422369,...,6.538022,6.207947,0.114257,0.101562,0.710931,0.723626,0.063476,0.101562,0.063476,0.050781
716129,50,56,43,Wyoming,Washakie County,12,0,7805,50.775144,49.224856,...,6.559898,6.265215,0.089686,0.115311,0.691864,0.755926,0.089686,0.102498,0.051249,0.025625
716319,50,56,45,Wyoming,Weston County,10,0,6968,52.525832,47.474168,...,2.009185,1.693456,0.086108,0.100459,0.330080,0.258324,0.028703,0.086108,0.028703,0.028703
716338,50,56,45,Wyoming,Weston County,11,0,6924,52.383016,47.616984,...,1.906412,1.617562,0.086655,0.115540,0.346620,0.317735,0.072213,0.014443,0.014443,0.000000


* Columns of demographic ratios are produced
* Ratios are demographic / county population


In [246]:
county_dem_ratio = get_df_with_geofips(county_dem_ratio, 'STATE', 'COUNTY')

In [247]:
cols_to_remove =  ['AGEGRP', 'STATE', 'COUNTY', 'SUMLEV']
county_dem_ratio = remove_col(county_dem_ratio, cols_to_remove)
display(county_dem_ratio)

,GeoFIPS,STNAME,CTYNAME,YEAR,TOT_POP,TOT_MALE_ratio,TOT_FEMALE_ratio,WA_MALE_ratio,WA_FEMALE_ratio,BA_MALE_ratio,...,HWAC_MALE_ratio,HWAC_FEMALE_ratio,HBAC_MALE_ratio,HBAC_FEMALE_ratio,HIAC_MALE_ratio,HIAC_FEMALE_ratio,HAAC_MALE_ratio,HAAC_FEMALE_ratio,HNAC_MALE_ratio,HNAC_FEMALE_ratio
171,01001,Alabama,Autauga County,10,55390,48.763315,51.236685,37.689114,39.185774,9.277848,...,1.261961,1.160859,0.200397,0.140820,0.070410,0.074021,0.021665,0.027081,0.030691,0.027081
190,01001,Alabama,Autauga County,11,55533,48.630544,51.369456,37.665892,39.164101,9.212540,...,1.375759,1.210091,0.162066,0.135055,0.070229,0.057623,0.036015,0.028812,0.034214,0.023410
209,01001,Alabama,Autauga County,12,55869,48.492008,51.507992,37.369561,38.892767,9.373714,...,1.392543,1.229662,0.159301,0.166461,0.071596,0.048327,0.026849,0.034008,0.028638,0.019689
399,01003,Alabama,Baldwin County,10,212521,48.568377,51.431623,42.518151,44.802631,4.366157,...,2.243072,2.000743,0.121870,0.127046,0.112930,0.076228,0.019763,0.026350,0.020704,0.018822
418,01003,Alabama,Baldwin County,11,217855,48.519428,51.480572,42.511303,44.899130,4.301944,...,2.281334,2.053201,0.120722,0.120722,0.118427,0.085378,0.028459,0.025705,0.022951,0.014230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716110,56043,Wyoming,Washakie County,11,7877,50.577631,49.422369,48.000508,46.591342,0.317380,...,6.538022,6.207947,0.114257,0.101562,0.710931,0.723626,0.063476,0.101562,0.063476,0.050781
716129,56043,Wyoming,Washakie County,12,7805,50.775144,49.224856,48.161435,46.354901,0.320307,...,6.559898,6.265215,0.089686,0.115311,0.691864,0.755926,0.089686,0.102498,0.051249,0.025625
716319,56045,Wyoming,Weston County,10,6968,52.525832,47.474168,49.469001,44.646958,0.416188,...,2.009185,1.693456,0.086108,0.100459,0.330080,0.258324,0.028703,0.086108,0.028703,0.028703
716338,56045,Wyoming,Weston County,11,6924,52.383016,47.616984,49.119006,44.381860,0.462161,...,1.906412,1.617562,0.086655,0.115540,0.346620,0.317735,0.072213,0.014443,0.014443,0.000000


* GeoFIPS column added
* Redundant column removes and columns with lack of data variety

In [248]:
county_dem_ratio_17 = county_dem_ratio.loc[county_demographic.YEAR == 10]
county_dem_ratio_18 = county_dem_ratio.loc[county_demographic.YEAR == 11]
county_dem_ratio_19 = county_dem_ratio.loc[county_demographic.YEAR == 12]

county_dem_ratio_17.drop('YEAR', axis=1, inplace=True)
county_dem_ratio_18.drop('YEAR', axis=1, inplace=True)
county_dem_ratio_19.drop('YEAR', axis=1, inplace=True)

/home/wkc/.local/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


# Merge data

## merge poverty rates for 2017-2019 into one dataframe

In [249]:
pov_rate_2017.rename(columns={'Poverty Percent, All Ages':'Poverty Percent, All Ages 2017'}, inplace=True)
pov_rate_2018.rename(columns={'Poverty Percent, All Ages':'Poverty Percent, All Ages 2018'}, inplace=True)
pov_rate_2019.rename(columns={'Poverty Percent, All Ages':'Poverty Percent, All Ages 2019'}, inplace=True)

pov_rate_2017_2018 = pd.merge(pov_rate_2017, pov_rate_2018, how='inner')
pov_rate_2017_2019 = pd.merge(pov_rate_2017_2018, pov_rate_2019, how='inner')
pov_rate_2017_2019

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Percent, All Ages 2017","Poverty Percent, All Ages 2018","Poverty Percent, All Ages 2019"
0,01,001,AL,Autauga County,13.400000,13.800000,12.100000
1,01,003,AL,Baldwin County,10.100000,9.800000,10.100000
2,01,005,AL,Barbour County,33.400002,30.900000,27.100000
3,01,007,AL,Bibb County,20.200001,21.799999,20.299999
4,01,009,AL,Blount County,12.800000,13.200000,16.299999
...,...,...,...,...,...,...,...
3001,56,037,WY,Sweetwater County,10.000000,8.400000,8.300000
3002,56,039,WY,Teton County,5.800000,6.300000,6.000000
3003,56,041,WY,Uinta County,10.200000,10.000000,8.500000
3004,56,043,WY,Washakie County,10.700000,11.900000,11.100000


In [250]:
state_code = pov_rate_2017_2019['State FIPS Code'].astype(str).values
county_code = pov_rate_2017_2019['County FIPS Code'].astype(str).values
geofips = []
for stateID, countyID in zip(state_code, county_code):
    id = ''
    id += stateID
    id += countyID
    geofips.append(id)
pov_rate_2017_2019.insert(loc=0, column='GeoFIPS', value=geofips)

In [251]:
pov_rate_2017_2019.drop(['State FIPS Code', 'County FIPS Code'], axis=1, inplace=True)
display(pov_rate_2017_2019)

,GeoFIPS,Postal Code,Name,"Poverty Percent, All Ages 2017","Poverty Percent, All Ages 2018","Poverty Percent, All Ages 2019"
0,01001,AL,Autauga County,13.400000,13.800000,12.100000
1,01003,AL,Baldwin County,10.100000,9.800000,10.100000
2,01005,AL,Barbour County,33.400002,30.900000,27.100000
3,01007,AL,Bibb County,20.200001,21.799999,20.299999
4,01009,AL,Blount County,12.800000,13.200000,16.299999
...,...,...,...,...,...,...
3001,56037,WY,Sweetwater County,10.000000,8.400000,8.300000
3002,56039,WY,Teton County,5.800000,6.300000,6.000000
3003,56041,WY,Uinta County,10.200000,10.000000,8.500000
3004,56043,WY,Washakie County,10.700000,11.900000,11.100000


## merge education and vehicle rates for 2017-2019 into one dataframe

In [252]:
educ_vehicle_ratios = educ_vehicle_2017.columns[4:]
for ratio in educ_vehicle_ratios:
    educ_vehicle_2017.rename(columns={ratio:ratio + '_2017'}, inplace=True)
    educ_vehicle_2018.rename(columns={ratio:ratio + '_2018'}, inplace=True)
    educ_vehicle_2019.rename(columns={ratio:ratio + '_2019'}, inplace=True)

educ_vehicle_2017.drop('POP', axis=1, inplace=True)
educ_vehicle_2018.drop('POP', axis=1, inplace=True)
educ_vehicle_2019.drop('POP', axis=1, inplace=True)

In [253]:
educ_vehicle_2017_2018 = pd.merge(educ_vehicle_2017, educ_vehicle_2018, how='inner')
educ_vehicle_2017_2019 = pd.merge(educ_vehicle_2017_2018, educ_vehicle_2019, how='inner')
educ_vehicle_2017_2019

,NAME,state,county,EDU_TOT_ratio_2017,LESS_HS_TOT_ratio_2017,HS_TOT_ratio_2017,COL_OR_ASSOC_TOT_ratio_2017,BACH_TOT_ratio_2017,GRAD_TOT_ratio_2017,VEHICLE_TOT_ratio_2017,...,BACH_TOT_ratio_2018,GRAD_TOT_ratio_2018,VEHICLE_TOT_ratio_2018,EDU_TOT_ratio_2019,LESS_HS_TOT_ratio_2019,HS_TOT_ratio_2019,COL_OR_ASSOC_TOT_ratio_2019,BACH_TOT_ratio_2019,GRAD_TOT_ratio_2019,VEHICLE_TOT_ratio_2019
0,"St. Lawrence County, New York",36,89,66.185928,8.359560,23.588116,18.327358,7.552247,8.358647,32.766846,...,7.361611,7.866947,35.461420,67.044737,8.564136,23.762762,19.480230,8.060145,7.177464,31.232597
1,"Saratoga County, New York",36,91,71.525086,4.635684,16.980541,19.820419,16.481561,13.606880,45.984887,...,16.692083,13.376173,46.076042,72.095118,4.492676,17.754924,19.172290,17.466056,13.209172,45.281320
2,"Schenectady County, New York",36,93,69.072735,6.482821,20.662745,22.667695,11.062900,8.196574,38.755504,...,11.787576,10.642420,38.725459,70.183324,5.047682,20.198456,20.343982,14.569959,10.023245,38.615831
3,"Steuben County, New York",36,101,70.638028,6.761459,26.921199,21.483990,7.738806,7.732574,36.959525,...,8.302017,9.023341,38.738569,71.044989,6.093585,25.791841,21.817171,8.456788,8.885604,37.225175
4,"Suffolk County, New York",36,103,69.456038,6.873693,18.559191,18.702062,13.936407,11.384685,41.375382,...,13.946120,11.819717,41.403545,70.148605,6.397598,18.389192,18.517663,14.495114,12.349037,42.622211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,"Putnam County, New York",36,79,71.933993,5.124694,20.737392,18.218338,15.030758,12.822810,42.683970,...,16.365328,13.018242,40.700967,73.443857,5.721115,19.355167,20.396664,15.500407,12.470504,43.144833
832,"Queens County, New York",36,81,71.878018,12.930820,20.936054,15.809711,14.245424,7.956009,16.619308,...,15.182197,8.990191,17.077931,72.484558,12.328372,19.812340,16.300805,14.831103,9.211938,17.209159
833,"Rensselaer County, New York",36,83,69.525175,5.694269,19.865767,21.884274,12.611287,9.469578,41.146492,...,13.420554,8.401801,42.175211,70.327759,5.902441,19.919478,19.870963,13.165820,11.469058,41.341659
834,"Richmond County, New York",36,85,69.678470,8.140650,21.009139,17.452832,13.788069,9.287779,26.693683,...,14.675154,9.263113,26.658042,70.041143,7.701258,20.426427,16.545240,15.103446,10.264773,27.038096


In [254]:
educ_vehicle_2017_2019 = get_df_with_geofips(educ_vehicle_2017_2019, 'state', 'county')
educ_vehicle_2017_2019

,GeoFIPS,NAME,state,county,EDU_TOT_ratio_2017,LESS_HS_TOT_ratio_2017,HS_TOT_ratio_2017,COL_OR_ASSOC_TOT_ratio_2017,BACH_TOT_ratio_2017,GRAD_TOT_ratio_2017,...,BACH_TOT_ratio_2018,GRAD_TOT_ratio_2018,VEHICLE_TOT_ratio_2018,EDU_TOT_ratio_2019,LESS_HS_TOT_ratio_2019,HS_TOT_ratio_2019,COL_OR_ASSOC_TOT_ratio_2019,BACH_TOT_ratio_2019,GRAD_TOT_ratio_2019,VEHICLE_TOT_ratio_2019
0,36089,"St. Lawrence County, New York",36,89,66.185928,8.359560,23.588116,18.327358,7.552247,8.358647,...,7.361611,7.866947,35.461420,67.044737,8.564136,23.762762,19.480230,8.060145,7.177464,31.232597
1,36091,"Saratoga County, New York",36,91,71.525086,4.635684,16.980541,19.820419,16.481561,13.606880,...,16.692083,13.376173,46.076042,72.095118,4.492676,17.754924,19.172290,17.466056,13.209172,45.281320
2,36093,"Schenectady County, New York",36,93,69.072735,6.482821,20.662745,22.667695,11.062900,8.196574,...,11.787576,10.642420,38.725459,70.183324,5.047682,20.198456,20.343982,14.569959,10.023245,38.615831
3,36101,"Steuben County, New York",36,101,70.638028,6.761459,26.921199,21.483990,7.738806,7.732574,...,8.302017,9.023341,38.738569,71.044989,6.093585,25.791841,21.817171,8.456788,8.885604,37.225175
4,36103,"Suffolk County, New York",36,103,69.456038,6.873693,18.559191,18.702062,13.936407,11.384685,...,13.946120,11.819717,41.403545,70.148605,6.397598,18.389192,18.517663,14.495114,12.349037,42.622211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,36079,"Putnam County, New York",36,79,71.933993,5.124694,20.737392,18.218338,15.030758,12.822810,...,16.365328,13.018242,40.700967,73.443857,5.721115,19.355167,20.396664,15.500407,12.470504,43.144833
832,36081,"Queens County, New York",36,81,71.878018,12.930820,20.936054,15.809711,14.245424,7.956009,...,15.182197,8.990191,17.077931,72.484558,12.328372,19.812340,16.300805,14.831103,9.211938,17.209159
833,36083,"Rensselaer County, New York",36,83,69.525175,5.694269,19.865767,21.884274,12.611287,9.469578,...,13.420554,8.401801,42.175211,70.327759,5.902441,19.919478,19.870963,13.165820,11.469058,41.341659
834,36085,"Richmond County, New York",36,85,69.678470,8.140650,21.009139,17.452832,13.788069,9.287779,...,14.675154,9.263113,26.658042,70.041143,7.701258,20.426427,16.545240,15.103446,10.264773,27.038096


## Merge demographic years as one spanning 2017-2019


In [255]:
# add suffix to each column name then merge
demographic = county_dem_ratio_17.columns.values[3:]

for dem in demographic:
    county_dem_ratio_17.rename(columns={dem:dem + '_2017'}, inplace=True)
    county_dem_ratio_18.rename(columns={dem:dem + '_2018'}, inplace=True)
    county_dem_ratio_19.rename(columns={dem:dem + '_2019'}, inplace=True)

/home/wkc/.local/lib/python3.8/site-packages/pandas/core/frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [256]:
county_dem_ratio_2017_2018 = pd.merge(county_dem_ratio_17, county_dem_ratio_18, how='inner')
county_dem_ratio_2017_2019 = pd.merge(county_dem_ratio_2017_2018, county_dem_ratio_19, how='inner')
county_dem_ratio_2017_2019.shape

(3142, 222)

## Merge avg northeast aqi data

In [257]:
avg_aqi_2017_2018 = pd.merge(average_aqi_2017, average_aqi_2018, how='inner')
avg_aqi_2017_2019 = pd.merge(avg_aqi_2017_2018, average_aqi_2019, how='inner')
avg_aqi_2017_2019


,GeoFIPS,State Name,county Name,AQI_2017,AQI_2018,AQI_2019
0,01003,Alabama,Baldwin,37.503704,36.829630,38.291513
1,01027,Alabama,Clay,32.457627,28.990909,31.149533
2,01033,Alabama,Colbert,36.155477,35.635379,35.520913
3,01049,Alabama,DeKalb,38.239554,36.720000,39.711911
4,01051,Alabama,Elmore,34.371681,35.328829,38.421053
...,...,...,...,...,...,...
1025,72113,Puerto Rico,Ponce,33.734440,46.448179,39.233618
1026,78010,Virgin Islands,St Croix,24.346154,29.090909,24.877193
1027,78020,Virgin Islands,St John,26.223881,21.000000,21.752475
1028,80002,Country Of Mexico,BAJA CALIFORNIA NORTE,74.343490,85.000000,77.125000


# saved cleaned up datasets


In [258]:
data_path = r'../../data/processed/county_incomes_2017_2019'
county_incomes_2017_2019.to_csv(data_path, index=False)

data_path = r'../../data/processed/clean_edu_vehicle_2017_2019.csv'
educ_vehicle_2017_2019.to_csv(data_path, index=False)

data_path = r'../../data/processed/clean_pov_rate_2017_2019.csv'
pov_rate_2017_2019.to_csv(data_path, index=False)

data_path = r'../../data/processed/clean_UE_rate_2017_2019.csv'
UE_county_rate.to_csv(data_path, index=False)

data_path = r'../../data/processed/clean_avg_aqi_2017_2019'
avg_aqi_2017_2019.to_csv(data_path, index=False)

data_path = r'../../data/processed/clean_county_dem_ratio_2017_2019.csv'
county_dem_ratio_2017_2019.to_csv(data_path, index=False)
